In [1]:
import tensorflow as tf, numpy as np, shutil, random, os, csv
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import glob

In [2]:
# define variables
train_path='./data/td/training/'
validation_path='./data/td/training/'
reference_train_path=train_path + 'REFERENCE.csv'
reference_validation_path=validation_path + 'REFERENCE.csv'
train_dirs=os.listdir(train_path)
train_size=len(train_dirs)

batch_size= 128
timesteps = 1000

n_layers = 2
n_epochs = 200
n_classes = 2
n_units = 30
n_validation_files = 300

x_placeholder = tf.placeholder('float32', [None, batch_size, 1])
y_placeholder = tf.placeholder('int32', [batch_size, n_classes])

validation_x_placeholder = tf.placeholder('float32', [None, n_validation_files, 1])
validation_y_placeholder = tf.placeholder('int32', [n_validation_files, n_classes])

In [3]:
def get_batch(path, batch_size):
    filenames = glob.glob(os.path.join(path, 'e*.npy'))
    return random.sample(filenames, batch_size)

def get_batch_max_len(batch_files):
    batch_max_len=0
    for f in batch_files:
        batch_max_len=max(batch_max_len, np.load(f).shape[0])
    return batch_max_len

def get_reference(reference_path, reference):
    with open(reference_path) as csvfile:
        spamreader=csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            reference[row[0]]=row[1]

def generate_batch(path, b_size):
    if b_size == None:
        b_size = batch_size

    batch_files=get_batch(path, b_size)
   
    max_seq=get_batch_max_len(batch_files)

    batchX=np.zeros((b_size, 1, max_seq), dtype='float32')
    i=0
    for f in batch_files:
        arr=np.load(f)
        arr=np.asmatrix(arr)
        batchX[i][0][0:arr.shape[1]]=arr

    i=0
    batchY=np.zeros(b_size, dtype='int32')
    for f in batch_files:
        batchY[i] = reference_train[f.split('/')[-1].split('.')[0]]
        i += 1
    
    # reshape batchX
    batchX = np.swapaxes(batchX, 1, 2)
    batchX = np.swapaxes(batchX, 0, 1)
    
    # convert batchY to onehot
    y_onehot = np.zeros((b_size, 2))
    y_onehot[np.where(batchY==1)] = np.array([0,1])
    y_onehot[np.where(batchY==-1)] = np.array([1,0])
    
    batchX = batchX[0:timesteps,:,:]
    print('batchX, y_onehot, sizes', batchX.shape, y_onehot.shape)
    
    return batchX, y_onehot

In [4]:
def RNN(t_input_data, v_input_data, cell_name, n_layers):
    w = tf.Variable(tf.random_normal([n_units, n_classes]))
    b = tf.Variable(tf.random_normal([n_classes]))
    
    if cell_name == 'basic':
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.BasicRNNCell(n_units, activation=tf.nn.relu) for _ in range(n_layers)])
    elif cell_name == 'LSTM':
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(n_units) for _ in range(n_layers)])
    else:
        input_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(n_units) for _ in range(n_layers)])

    t_outputs, t_states = tf.nn.dynamic_rnn(cell=input_cell, 
                                            inputs=t_input_data,
                                            dtype=tf.float32,
                                            time_major=True)
    
    v_outputs, v_states = tf.nn.dynamic_rnn(cell=input_cell, 
                                            inputs=v_input_data,
                                            dtype=tf.float32,
                                            time_major=True)
    

    return tf.matmul(t_outputs[-1], w) + b, tf.matmul(v_outputs[-1], w) + b

def get_optimizer(opt_name, cost):
    if opt_name == 'adam':
        return tf.train.AdamOptimizer().minimize(cost)
    elif opt_name == 'ada':
        return tf.train.AdadeltaOptimizer().minimize(cost)
    elif opt_name == 'gradient':
        return tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    else:
        return tf.train.MomentumOptimizer(0.001, 0.4).minimize(cost)

def train_neural_network(cell_name, layers, opt_name):
    print('Using Cell', cell_name, 'layers', layers, 'opt_name', opt_name)
    
    t_prediction, v_prediction = RNN(x_placeholder, validation_x_placeholder, cell_name, layers)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=t_prediction,labels=tf.reshape(y_placeholder, [batch_size, n_classes])))
    optimizer = get_optimizer(opt_name, cost)
    correct = tf.equal(tf.argmax(v_prediction, 1), tf.argmax(validation_y_placeholder, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float32'))
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        max_accuracy = 0
        for epoch in range(n_epochs):            
            batchX, batchY = generate_batch(train_path, None)
            validation_batchX, validation_batchY = generate_batch(validation_path, n_validation_files) 
            _, epoch_loss, epoch_validation_accuracy = sess.run([optimizer, cost, accuracy], 
                                                                feed_dict={x_placeholder: batchX, 
                                                                    y_placeholder: batchY, 
                                                                    validation_x_placeholder: validation_batchX, 
                                                                    validation_y_placeholder: validation_batchY})
            max_accuracy = max(max_accuracy, epoch_validation_accuracy)
            print('Epoch', epoch+1, 'loss:', epoch_loss, 'Validation Accuracy:', epoch_validation_accuracy, 'max_accuracy', max_accuracy)

In [5]:
# read in reference train/validation file
reference_train, reference_validation = {}, {}
get_reference(reference_train_path, reference_train)
get_reference(reference_validation_path, reference_validation)

train_neural_network('LSTM', n_layers, 'adam')

('Using Cell', 'LSTM', 'layers', 2, 'opt_name', 'adam')
('batchX, y_onehot, sizes', (1000, 128, 1), (128, 2))
('batchX, y_onehot, sizes', (1000, 300, 1), (300, 2))
('Epoch', 1, 'loss:', 0.62618053, 'Validation Accuracy:', 0.90333331, 'max_accuracy', 0.90333331)
('batchX, y_onehot, sizes', (1000, 128, 1), (128, 2))
('batchX, y_onehot, sizes', (1000, 300, 1), (300, 2))
('Epoch', 2, 'loss:', 0.6154741, 'Validation Accuracy:', 0.91000003, 'max_accuracy', 0.91000003)
('batchX, y_onehot, sizes', (1000, 128, 1), (128, 2))
('batchX, y_onehot, sizes', (1000, 300, 1), (300, 2))
('Epoch', 3, 'loss:', 0.59256232, 'Validation Accuracy:', 0.87333333, 'max_accuracy', 0.91000003)
('batchX, y_onehot, sizes', (1000, 128, 1), (128, 2))
('batchX, y_onehot, sizes', (1000, 300, 1), (300, 2))
('Epoch', 4, 'loss:', 0.53830338, 'Validation Accuracy:', 0.89666665, 'max_accuracy', 0.91000003)
('batchX, y_onehot, sizes', (1000, 128, 1), (128, 2))
('batchX, y_onehot, sizes', (1000, 300, 1), (300, 2))
('Epoch', 5, 

KeyboardInterrupt: 